Algorithmic Methods for Data Mining
==============================


## Homework 3 - Airbnb Search Engine
### Group #17 - Giulia Scialanga, Guilherme Nicchio, Marco Minici
##### 26/11/2018

The goal of this project consists in buiding a search engine over a data base of Airbnb houses. 

The code returns the houses of the data base which matches the descriptions entered by an user query. 

User query means the sentence an user enter in a search field, for example: "A beuatiful house with beach and garden"

For this project was used the following libraries:

In [1]:
from os.path import isdir
from os import mkdir
import pandas as pd
import numpy as np
import csv
import math
from os import listdir
from os.path import isfile
from os import remove
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from geopy import distance
import folium
import os
from heapq import nlargest
from nltk import SnowballStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
import string
import pickle
import inflect

The functions created for the purpose of this work are in the file **airbnb_search_engine.py** located in the repository. For displaying the results of this work let's import it and assign the main class of it to a variable.

In [13]:
import airbnb_search_engine
airbnb = airbnb_search_engine.AirBnbPy()

# The Data

The data used is a csv file with the following columns.

In [14]:
data = airbnb.loadData()
data.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url
1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...
2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...
3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...
4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...
5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...


The first goal on the process is to match the user query with descriptions and tittles of the houses.

## Cleaning the data

#### Cleaning process:

It was noticed that the data base has some "noise" within it, for example empty cells (NA) in location coordinates, title and description which is essential for the concept of search engine built and the houses of the respective residences won't produce any match to an entered query.

It was also noticed duplicated cells, that sometimes are system error or a house entered in the system twice by the owner, in order to avoid a duplicated outcome for the query the duplicated cells will be dropped out of the data base.

Cleaning process:

-Delete all rows with (lat,long) equal to NA since if an house is not locatable it is useless for a customer.

-Delete rows with both description and title equal to NA since it wouldn't be possible to match the user query.

-Retain all others rows, maybe later penalizing records with NA values for other columns(e.g.:bedrooms_count).

#### 1.1.1 Load the data.csv

- read csv file with pandas csv read.

#### 1.1.2 Call cleandata()

- Gets just data which has not null in latitude column;

- Drop NA's from description and title;

- Drop all duplicated rows;

- Returns the data.


## Pre-processing the data and creating Tsv files for each house
At this point there is a data frame with duplicated removed and empty fields removed.
The next step is to go through each row of the data frame process it and write a tsv file with it.

Before diving into text and feature extraction, the first step will be further treating the data in order to obtain better features. We will achieve this by doing some of the basic _*pre-processing*_ steps on our data, for this it was used the library NLTK which has many features for natural language processing.

### Pre-processing

###### Lower case
The first pre-processing step is to transform our descriptions and titles into lower case. This avoids having multiple copies of the same words. For example, while calculating the word count, ‘House’ and ‘house’ would be taken as different words.

##### Tokenization
Tokenization refers to dividing the text into a sequence of words or sentences.

##### Removing Punctuation and Stop Words
The next step is to remove punctuation and stopwords, as it doesn’t add any extra information while treating text data. Therefore removing all instances of it will help us reduce the size of the data.

##### Stemming
Stemming refers to the removal of suffices, like “ing”, “ly”, “s”, etc. by a simple rule-based approach. For this purpose, it will be used SnowballStemmer from the NLTK library.

##### Removing non-enlish characters
When handling the data was noticed non-english texts, for example descriptions entered in chinese language can bring problematics when processing the text. In order to solve this issue was created a function to test if the description an title are indeed using english characters, therefore, the data just proceed for process and creating tsv file if the english test function returns true for the test.

### Creating TSV files
After pre processing the text we join the description and title text and replace then in the original field, proceeding to the tsv file creation. The treated data has 18259 rolls which will result in the same number of tsv files for the analysis of the search engine with the title and description properly pre-processed

    1. calls createAllReviews
       Calls CreateTSV to all lines of the dataframe com .apply(lambda)
        2.CreateTSV (inside a line of the dataframe the following will happen)
            If description is empty (.isna) writes "NaN" else calls _nltkProcess
            The same for title
                3. _nltkProcess
                    String lower
                    4. calls _setupNltk
                        Lazy initialization of objects needed to preprocess strings
                        tokenizer = RegexpTokenizer(r'\w+')
                        stopwords = set(stopwords.words('english'))
                        stemmer = SnowballStemmer('english') 
                    Join the result and return
                5. Calls isEnglish() to test over each word int the line if they are in english and not chinese
                    Try to encode into english, if fails return FALSE for test "is english"
                    If succeeds return TRUE for the test
                iI returned TRUE goes on to create the file, if False returns from CreateTSV and don't create the file
                open the file to write using with open(self.dir_path+self.review_dir+"doc_"+str(x.name)+".tsv", 'w') as file:
                try to write it
                    except (if fails to write) save the index (x.name) to delete the file afterwards

# Building the Search Engine

This section consists in:

- Build the encoding for the words in the corpus. To each word is associated a unique int ID.

- Create the reverted index. To each word ID is associated the set of documents which contains that word.

- Realize an interface to perform queries to the search engine.

### Building the Encoding
At this point the tsv files are created and pre-processed. The next step is to create a dictionary for the vocabulary of words present in all the documents, in this dictionary each word corresponds to a single number. This process will help analyse which documents have the respective word.

Example of what is the target at this point: {"car":1, "house":2}

This step was approached browsing through every file's title and description and adding its words into an empty dictionary. But before adding it is checked if the word isn't yet in the dictionary, when adding the word to the dictionary its key value is a counter of how many unique words were collected so far. 

    1.BuildingEncoding
        create an empty dictionary to store words
        create a counter to store the index of how many single words we have (vocabulary size)
        for each file in "listdir(directory address)"
            goes to the title, strip its words and create a set of it to remove duplicates
            do the same for the description
            union both sets
            for each word in the united set check if it is not dictionary already
                if it isnt add it to the dictionary as a key
                set the counter as the value of the word in the dictionary
                increase the counter +1

### Reverse Index
Creating a dictionary of words as keys and documents as values.

After this step can say in for each number (word) which documents have it example 

 
{

term_id_1:[document_1, document_2, document_4],

term_id_2:[document_1, document_3, document_5, document_6],

...}


In this step it is created a new dictionary which the keys are the the vocabulary numbers of the previous dictionary. Then it is checkd for each document if the title and the description has the respective word, if there is the document name is appended as a value to the respective keyword.

In [28]:
reverted_index = airbnb.RI

Documents containing the word related to the index 1.

In [38]:
reverted_index.get(1)

['doc_1',
 'doc_10000',
 'doc_10006',
 'doc_10012',
 'doc_10015',
 'doc_10019',
 'doc_10021',
 'doc_10028',
 'doc_10030',
 'doc_10034',
 'doc_10035',
 'doc_10036',
 'doc_10037',
 'doc_10039',
 'doc_1005',
 'doc_10054',
 'doc_10056',
 'doc_10057',
 'doc_10062',
 'doc_10067',
 'doc_10068',
 'doc_10071',
 'doc_10076',
 'doc_10077',
 'doc_10078',
 'doc_10081',
 'doc_10084',
 'doc_10087',
 'doc_10097',
 'doc_1010',
 'doc_10101',
 'doc_10102',
 'doc_10104',
 'doc_10112',
 'doc_10116',
 'doc_10119',
 'doc_10124',
 'doc_10130',
 'doc_10132',
 'doc_10136',
 'doc_10137',
 'doc_10139',
 'doc_10141',
 'doc_10143',
 'doc_10144',
 'doc_10148',
 'doc_10160',
 'doc_10161',
 'doc_10162',
 'doc_10164',
 'doc_10169',
 'doc_1017',
 'doc_10172',
 'doc_10175',
 'doc_10176',
 'doc_10177',
 'doc_10178',
 'doc_1018',
 'doc_10183',
 'doc_10188',
 'doc_1019',
 'doc_10190',
 'doc_10199',
 'doc_10202',
 'doc_10203',
 'doc_1021',
 'doc_10211',
 'doc_10216',
 'doc_1023',
 'doc_10234',
 'doc_10239',
 'doc_10241',
 'd

In [39]:
len(reverted_index)

12385

We have 12385 different words we have in all the documents.

    create a new dictionary with the keys being the numbers at the size of the vocabulary
        this is our target dictionary, we want to say what documents have this word
    for each file in "listdir(directory address)"
        goes to the title, strip its words and create a set of it to remove duplicates
        do the same for the description
        union both sets
        for each word in the set
            get which number this word has in the vocabulary dictionary
            goes to the new dictionary and append the name of the document to the key

# Conjunctive query

At this moment, we narrow out interest on the description and title of each document. It means that the first Search Engine will evaluate queries with respect to the aforementioned information.

Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output is a table of documents with title, description, city and url.

Our first goal is to get the query and process it in order to be in the same standard as the description and title.

### Query processing

After the user enter a query, which is considered a sentence which will be acquired in string format and apply the same process of Tokenization, eliminating stopwords and stemming the words, which were described in the **Pre-processing**.

For example, a query **"a beautiful house with garden and beach"** turns into a list **['beauti', 'hous', 'garden', 'beach']**

Now that we have the query properly formated its possible to go to the dictionary created in the previous section and check which documents have the words of the query.

In this step we go through each word in the query list and append the set of documents of present on the dictionary values to a new list. At the end of the preccess this new list has all sets of documents which contain the query words

As it is wanted a document that has all the words in it we do an intersection of this sets, which correspond to the group of documents that have all the words.

### Example

First let's create the term encoding and the reverted index dictionary to perform the query.

In [15]:
airbnb.buildEncoding(fileName = "term_encoding.pickle")
airbnb.createRevertedIndex(fileName = "reverted_index.pickle")

Imputing the query and performing the search.

In [18]:
result,_ = airbnb.query(str(input("Enter here your search: ")))
print(result)

Enter here your search: a beautiful house with garden and beach
Query processed:  ['beauti', 'hous', 'garden', 'beach']
Empty DataFrame
Columns: [title, description, city, url]
Index: []


Note that the query function returned a empty data frame, this means that there are no results matching this query. Let's try to simplify the query.

In [20]:
result,_ = airbnb.query(str(input("Enter here your search: ")))
result.head()

Enter here your search: a house at the beach
Query processed:  ['hous', 'beach']


,title,description,city,url
4067,Bring the whole family & the dog. 5 min to beach!,Our beach house has everything you need: comfo...,Port Aransas,https://www.airbnb.com/rooms/13686358?location...
1696,Unwind & Play | Sun-Drenched Galveston Beach H...,"Welcome to Camp Lafitte, a classic Galveston b...",Galveston,https://www.airbnb.com/rooms/16484229?location...
5839,"League City, TX Rental 3 bedroom, 2 baths.","1600 sq ft house in League City, TX, a great l...",League City,https://www.airbnb.com/rooms/16603529?location...
13351,"Historic Cole St House, walk to water, Cole Park",1600 Square foot historic house in a great nei...,Corpus Christi,https://www.airbnb.com/rooms/14211883?location...
17090,The COVE & RV,PIRATES COVE and RV is a beautiful beachfront ...,Crystal Beach,https://www.airbnb.com/rooms/6870793?location=...


In [21]:
len(result)

320

This query matched 320 houses descriptions.

## 2.4 Conjunctive query & Ranking score

Let's improve the Search Engine, given a query, now let's get the top 5 documents related to the query.

For this it will be find all the documents that contains all the words in the query, as before. But afterwards they will be sorted by their similarity with the query

To compute the simalarity will be used the tfIdf score, and the Cosine similarity.

The TF-IDF weight function ( term frequency-inverse document frequency ) is a function used in information retrieval to measure the importance of a term with respect to a document or a collection of documents. This function increases proportionally to the number of times the term is contained in the document, but grows in an inversely proportional manner with the frequency of the term in the collection. The idea behind this behavior is to give more importance to the terms that appear in the document, but that are generally infrequent.

Cosine similarity calculates similarity by measuring the cosine of angle between two vectors. This is calculated as:

\begin{align}
cos(\pmb x, \pmb y) = \frac {\pmb x \cdot \pmb y}{||\pmb x|| \cdot ||\pmb y||}
\end{align}

With cosine similarity, we need to convert sentences into vectors. One way to do that is to use bag of words with either TF (term frequency) or TF-IDF (term frequency- inverse document frequency). The choice of TF or TF-IDF depends on application and is immaterial to how cosine similarity is actually performed — which just needs vectors. TF is good for text similarity in general, but TF-IDF is good for search query relevance.

# The computation was done as follows.
### INSERT INFO OF THE CODE FLOW

In [42]:
result = airbnb.rankedQuery('a beautiful house', k =  10)

Query processed:  ['beauti', 'hous']


100%|███████████████████████████████████████████████████████████████████████████| 18036/18036 [00:51<00:00, 349.26it/s]


[LOG]: The file tfIdfMatrix.pickle doesn't exist
[LOG]: A new tfidf matrix will be built and saved in persistent memory with the name:= tfIdfMatrix.pickle
[LOG]: The file term_encodingTFIDF.pickle doesn't exist
[LOG]: A new term encoding will be built and saved in persistent memory with the name:= term_encodingTFIDF.pickle
[LOG]: The file RI2.pickle doesn't exist
[LOG]: A new inverted index will be built and saved in persistent memory with the name:= RI2.pickle


In [43]:
result

,title,description,city,url,Similarity
0,Quiet place,Beautiful house,New Caney,https://www.airbnb.com/rooms/16743790?location...,0.701582
1,"Beautiful, New Beach House","Our house is close to restaurants and dining, ...",Crystal Beach,https://www.airbnb.com/rooms/15068378?location...,0.455250
2,My house is big and quite.,My house is a Mexican rustic decorate. Big roo...,San Antonio,https://www.airbnb.com/rooms/7378323?location=...,0.352277
3,"Beautiful home near the splash pad, park and p...",The house is one house away from a park and .7...,Katy,https://www.airbnb.com/rooms/19079555?location...,0.348313
4,"Beautiful home near the splash pad, park and p...",The house is one house away from a park and .7...,Katy,https://www.airbnb.com/rooms/19079555?location...,0.348313
5,House on the Creek,"A beautiful house on the Creek, conveniently l...",Plano,https://www.airbnb.com/rooms/17992009?location...,0.346690
6,House in wonderful neighborhood & beautiful views,"My place is good for couples, solo adventurers...",San Antonio,https://www.airbnb.com/rooms/17345573?location...,0.345514
7,House in wonderful neighborhood & beautiful views,"My place is good for couples, solo adventurers...",San Antonio,https://www.airbnb.com/rooms/17345573?location...,0.345514
8,1. Room in beautiful home,"A nice clean room in a quiet, beautiful house ...",San Antonio,https://www.airbnb.com/rooms/13915253?location...,0.343841
9,Quite Spacious Private Room in Beautiful House,Our house is in a small and quite family neigh...,Houston,https://www.airbnb.com/rooms/18784719?location...,0.339466


## 2.5 Now let's define a new score

Let's be more proactive and ask the user to input some information to improve the previous result for the entered query.

For example, would be interesting for the user choose minimal amount of rooms, a location desired for the house and the maximum price it is willing to pay.

Let's imagine that we provide a platform where the user input in the respective fields the amount of rooms, the price also the user is able to point a desired location in map.

With this is possible to compute the similarity of each house to the target inputed by the user.

**The workflow for the developed code go as follow**

For each document, it is collected the number of rooms, the price, the latitude and longitude and the similarity is a weighted mean of each individual distance, outputing a number between 1 and 0 where 0 is the maximum match.

The weight of each similarity in the calculation of the final similarity was defined through trial until the results were satisfactory.

\begin{align}
Similarity = \frac{weight rooms \times Rooms Similarity + weight loc \times Location Similarity + weight price \times Price Similarity}{3}
\end{align}

Where the Rooms Similarity is calculated as:

\begin{align}
RoomsSimilarity = \frac{1}{1 + RoomsDifference^{10}}
\end{align}

The Location Similarity was calculated using the library geopy and the function geodesic which allows to compute the precise ellipsoid distance from one point of coordinates to the other on earth.

\begin{align}
LocationSimilarity = \frac{1}{1 + 0.1 \times GeoDistance}
\end{align}

And the Price Similarity is calculated as:

\begin{align}
PriceSimilarity = \frac{1}{\frac{House Price}{Price Requested}}
\end{align}

Although if the number of rooms desired is smaller than the rooms of the house it shouldn't be a problem, thus in this cases the rooms similarity is set to 1.

The same annalogy goes for the price, if the house is cheaper than the requested price the price similarity is also set to 1.

The weights chosen for this search method were 0.15 for the price, 0.25 for the rooms and 0.6 for the location. This parameters have proven to satisfy for many different custom queries.

### Further improving

Let's allow the user to change the previous custom ranking by inputing what is more relevant for him.

The user might be very restrict with location and would not mind to pay more for example. Than the user can input the ranking by opting for price oriented or location oriented search.

Where the following weights would be applied if the user wants to improve.

In [ ]:
weightsPrice = [0.7, 0.1, 0.2]
weightsLoc = [0.1, 0.2, 0.7]

## Example

In [67]:
#interface with the user
querytext = str(input('Enter your search: '))

queryinput =[]
queryinput.append(int(input("How much would you pay at maximum? ")))
queryinput.append(int(input("How many rooms do you want? ")))

location = list(map(float, input("Give us a location: ").split(", ")))

for x in location:
    queryinput.append(x)

Enter your search: a beach house
How much would you pay at maximum? 300
How many rooms do you want? 3
Give us a location: 29.064693, -95.134938


In [ ]:
airbnb.customQuery(query= queryinput, textQuery= querytext)

# Bonus Step: Make a nice visualization!

This section consists in displaying the user a map with the locations of the search engine.

The user will input a latitude and longitude of a specific location and the radius of the search. On a platform the user could click on a map and the code acquire this location.

In [72]:
location =  list(map(float, input('Choose a location: ').split(', ')))
coordinates = (float(location[0]), float(location[1]))
max_distance = input("Insert the distance: ")

#Intialize the folium map
m = folium.Map(location = coordinates, zoom_start=13)

Choose a location: 32.779594, -96.800768
Insert the distance: 2000


Creating a circle with the radio of the search inputed by the user and adding it to the map.

In [74]:
folium.Circle(
    radius= float(max_distance),
    location=(coordinates),
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)

In this step we go through every document, retrieve the house location and price and save it in a new list.

In [79]:
dir_path = airbnb.dir_path
review_dir = airbnb.review_dir
l_temp =[]
for filePath in listdir(dir_path+review_dir):
    data = pd.read_csv(dir_path+review_dir+ filePath, delimiter = '\t',header = None, encoding = 'utf8')
    latitude = float(data.values[0][5])
    longitude = float(data.values[0][6])
    #title = str(data.values[0][7]).strip()
    average_rate_per_night = str(data.values[0][0]).strip()
    l_temp.append((latitude, longitude, average_rate_per_night))    

Now that we have the information of every house available, let's see which of those have a distance to the target location smaller than the radius inputed by the user and mark it on the map.

In [81]:
folium.Marker(location = coordinates,
              icon=folium.Icon(color='green',icon='home')
             ).add_to(m)
for i in l_temp: 
    if (distance.geodesic(coordinates, (i[0],i[1]), ellipsoid = (6377., 6356., 1 / 297.)).meters <= float(max_distance)):
        folium.Marker(location = (i[0],i[1]), popup = i[2], icon = folium.Icon(color ='blue', icon='home')
                     ).add_to(m)

Saving the results

In [83]:
m.save(os.path.join('results', r"C:\Users\guilh\Desktop\ADM\HW3\HW03\map.html"))

In [ ]:
m